## Consultado os arquivos na camada silver

## Criando diretório gold

In [0]:
SILVER_PATH = "/Volumes/workspace/default/delta/silver"
GOLD_PATH = "/Volumes/workspace/default/delta/gold"
# Criar diretórios se não existirem
dbutils.fs.mkdirs(GOLD_PATH)

## Consultado os arquivos da camada silver

In [0]:
display(dbutils.fs.ls(f"{SILVER_PATH}/flight/"))

In [0]:
# Ler da camada Bronze
df_delays_silver = spark.read.format("delta").load(f"{SILVER_PATH}/flight/registro_voos")
df_codigos_silver = spark.read.format("delta").load(f"{SILVER_PATH}/flight/codigos_voos")
print(df_delays_silver)
print(df_codigos_silver)

In [0]:
# Registrar como views temporárias
# todo: usar spark ao invés de view temporaria para otimizar o processamento
# https://docs.databricks.com/sql/language-manual/delta-optimizations.html
df_delays_silver.createOrReplaceTempView("df_delays_silver")
df_codigos_silver.createOrReplaceTempView("df_codigos_silver")


In [0]:
df_query = spark.sql("""
SELECT 
dv.date, dv.origin, dv.destination, dv.distance_km, dv.is_delayed, dv.delay_minutes, dv.delay_category,
CASE
        WHEN dc2.Country  <> dc.Country THEN "Internacional"
        WHEN dc2.Country = dc.Country THEN "Nacional"
        ELSE "ERRO"
    END AS nacionalidade_voo,
CASE
        WHEN dc2.State <> dc.State AND dc2.Country = dc.Country THEN True
        WHEN dc2.State <> dc.State AND dc2.Country <> dc.Country THEN False
        ELSE null
    END AS interestadual
FROM df_delays_silver dv 
LEFT JOIN df_codigos_silver dc ON dv.origin = dc.IATA 
LEFT JOIN df_codigos_silver dc2 ON dv.destination = dc2.IATA 
""")
display(df_query)
# Escrever na camada Gold
#df_query.write.format("delta").mode("overwrite").save(GOLD_PATH)

In [0]:
spark.sql("SELECT current_catalog()").show()


In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS workspace.gold;


In [0]:
df_query.write.format("delta").mode("overwrite").saveAsTable("gold.fato_voos")

In [0]:
df_codigos_silver.write.format("delta").mode("overwrite").saveAsTable("gold.codigos_voos")